In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
#from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
#from keras import initializers, regularizers, constraints, optimizers, layers
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error

from keras.optimizers import Adam
from pandas import read_csv
from datetime import datetime
from pandas import read_csv
from matplotlib import pyplot
from sklearn import metrics

In [5]:

# load data
dataset = pd.read_csv('karl7.csv',low_memory=False, parse_dates= ['Timestamp'])
dataset = dataset.set_index('Timestamp')
dataset1 = pd.DataFrame()
'''Resample data for every 1 minute'''
dataset1['States'] = dataset['States'].resample('60s').count()
# save to file

dataset1.to_csv('data.csv')
dataset1.shape

dataset = read_csv('data.csv', header=0, index_col=0)
dataset = pd.concat([dataset, dataset.shift(1)], axis=1)
dataset.dropna(inplace=True)
values = dataset.values
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
#pyplot.plot(scaled[:,1])


# split into train and test set

n_train_hours = 11 * 24 * 60
train = scaled[:n_train_hours, :]
test = scaled[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

# configure network
n_batch = 32
n_epoch = 1
n_neurons = 20

'''Define the model'''

model = Sequential()
model.add(LSTM(n_neurons, return_sequences=True, batch_input_shape=(n_batch, train_X.shape[1], train_X.shape[2])))
model.add(LSTM(n_neurons))
model.add(Dense(1))

'''compile the model'''
model.compile(loss= 'mse', optimizer= 'adam', metrics=['mean_squared_error'])
model.summary()



((15840, 1, 1), (15840,), (2920, 1, 1), (2920,))
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_5 (LSTM)                    (32, 1, 20)           1760        lstm_input_3[0][0]               
____________________________________________________________________________________________________
lstm_6 (LSTM)                    (32, 20)              3280        lstm_5[0][0]                     
____________________________________________________________________________________________________
dense_3 (Dense)                  (32, 1)               21          lstm_6[0][0]                     
Total params: 5061
____________________________________________________________________________________________________


In [46]:
for i in range(n_epoch):
    model.fit(train_X, train_y, validation_data=(test_X, test_y), batch_size=n_batch, verbose=1, shuffle=False)
   


Train on 15840 samples, validate on 2920 samples
Epoch 1/10
15840/15840 [==============================] - 1s - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0033 - val_mean_squared_error: 0.0033
Epoch 2/10
15840/15840 [==============================] - 1s - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0033 - val_mean_squared_error: 0.0033
Epoch 3/10
15840/15840 [==============================] - 1s - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0033 - val_mean_squared_error: 0.0033
Epoch 4/10
15840/15840 [==============================] - 1s - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0033 - val_mean_squared_error: 0.0033
Epoch 5/10
15840/15840 [==============================] - 1s - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0033 - val_mean_squared_error: 0.0033
Epoch 6/10
15840/15840 [==============================] - 1s - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0033 - val_mean_squared_error: 0.0033
Epoch 7/1

In [29]:
'''Predictor Error'''

yhat = model.predict(train_X, batch_size=n_batch)
for i in range(len(train_y)):
    e = train_y[i] - yhat[i]
    #print('Error: %f' % e)